In [2]:
import io
import os
import requests
import pandas as pd
import pyarrow.parquet as pq
import geopandas as gpd
import matplotlib.pyplot as plt

#### Yellow Taxi Raw Data

In [41]:
def read_yellow_data(yellow_path):
    df = pd.read_parquet(yellow_path)

    # Standardize just the airport fee column
    if 'Airport_fee' in df.columns:
        df.rename(columns={'Airport_fee': 'airport_fee'}, inplace=True)
        
    # inspecting date type and modify to suitable data type
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.floor('s').dt.tz_localize(None)
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime']).dt.floor('s').dt.tz_localize(None)
    
    df = df.drop_duplicates().reset_index(drop=True)
    
    # set pk
    df['trip_id'] = df.index
    return df

In [42]:
yellow_path = "data/yellow_tripdata/yellow_tripdata_2022-01.parquet"
df = read_yellow_data(yellow_path)
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,trip_id
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.50,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0,0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.00,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0,1
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.50,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0,2
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.00,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0,3
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.50,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,None,90,170,0,8.00,0.0,0.5,2.39,0.0,0.3,13.69,NaN,NaN,2463926
2463927,2,2022-01-31 23:44:22,2022-01-31 23:55:01,NaN,4.19,NaN,None,107,75,0,16.80,0.0,0.5,4.35,0.0,0.3,24.45,NaN,NaN,2463927
2463928,2,2022-01-31 23:39:00,2022-01-31 23:50:00,NaN,2.10,NaN,None,113,246,0,11.22,0.0,0.5,2.00,0.0,0.3,16.52,NaN,NaN,2463928
2463929,2,2022-01-31 23:36:42,2022-01-31 23:48:45,NaN,2.92,NaN,None,148,164,0,12.40,0.0,0.5,0.00,0.0,0.3,15.70,NaN,NaN,2463929


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2463931 entries, 0 to 2463930
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

#### location_dim

In [6]:
def create_location_dim(zones_path, zone_lookup_path):
    zones = gpd.read_file(zones_path)
    zone_lookup = pd.read_csv(zone_lookup_path)
    location_dim = zones.merge(zone_lookup, on='LocationID').drop(columns=['Borough', 'Zone']).reset_index(drop=True)
    location_dim = location_dim.drop_duplicates(subset=['LocationID'])
    return location_dim

In [7]:
zones_path = 'data/taxi_zones/taxi_zones.shp'
zone_lookup_path = 'data/taxi_zone_lookup.csv'
create_location_dim(zones_path, zone_lookup_path).head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,service_zone
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",EWR
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",Boro Zone
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.77 256767.698, 1026495.593 2...",Boro Zone
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",Yellow Zone
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.31 144283.336, 936046.565 144...",Boro Zone


In [8]:
create_location_dim(zones_path, zone_lookup_path).info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 260 entries, 0 to 262
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   OBJECTID      260 non-null    int32   
 1   Shape_Leng    260 non-null    float64 
 2   Shape_Area    260 non-null    float64 
 3   zone          260 non-null    object  
 4   LocationID    260 non-null    int32   
 5   borough       260 non-null    object  
 6   geometry      260 non-null    geometry
 7   service_zone  260 non-null    object  
dtypes: float64(2), geometry(1), int32(2), object(3)
memory usage: 16.2+ KB


In [9]:
# Plot to see the geo of the df
# location_dim.plot(column='borough', legend=True, figsize=(10, 10))
# plt.title('NYC Taxi Zones by Borough')
# plt.show()

#### pickup_datetime_dim

In [10]:
def create_pickup_datetime_dim(df):
    pickup_datetime_dim = df[['tpep_pickup_datetime']].drop_duplicates().reset_index(drop=True)
    pickup_datetime_dim['pick_hour'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.hour
    pickup_datetime_dim['pick_day'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.day
    pickup_datetime_dim['pick_month'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.month
    pickup_datetime_dim['pick_year'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.year
    pickup_datetime_dim['pick_weekday'] = pickup_datetime_dim['tpep_pickup_datetime'].dt.weekday
    pickup_datetime_dim['pickup_datetime_id'] = pickup_datetime_dim.index

    return pickup_datetime_dim
    

In [31]:
create_pickup_datetime_dim(df)

,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,pickup_datetime_id
0,2022-01-01 00:35:40,0,1,1,2022,5,0
1,2022-01-01 00:33:43,0,1,1,2022,5,1
2,2022-01-01 00:53:21,0,1,1,2022,5,2
3,2022-01-01 00:25:21,0,1,1,2022,5,3
4,2022-01-01 00:36:48,0,1,1,2022,5,4
...,...,...,...,...,...,...,...
1423517,2022-01-31 23:23:43,23,31,1,2022,0,1423517
1423518,2022-01-31 23:36:07,23,31,1,2022,0,1423518
1423519,2022-01-31 23:09:46,23,31,1,2022,0,1423519
1423520,2022-01-31 23:02:51,23,31,1,2022,0,1423520


In [38]:
create_pickup_datetime_dim(df).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423522 entries, 0 to 1423521
Data columns (total 7 columns):
 #   Column                Non-Null Count    Dtype         
---  ------                --------------    -----         
 0   tpep_pickup_datetime  1423522 non-null  datetime64[us]
 1   pick_hour             1423522 non-null  int32         
 2   pick_day              1423522 non-null  int32         
 3   pick_month            1423522 non-null  int32         
 4   pick_year             1423522 non-null  int32         
 5   pick_weekday          1423522 non-null  int32         
 6   pickup_datetime_id    1423522 non-null  int64         
dtypes: datetime64[us](1), int32(5), int64(1)
memory usage: 48.9 MB


#### dropoff_datetime_dim

In [12]:
def create_dropoff_datetime_dim(df):
    dropoff_datetime_dim = df[['tpep_dropoff_datetime']].drop_duplicates().reset_index(drop=True)
    dropoff_datetime_dim['drop_hour'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.hour
    dropoff_datetime_dim['drop_day'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.day
    dropoff_datetime_dim['drop_month'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.month
    dropoff_datetime_dim['drop_year'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.year
    dropoff_datetime_dim['drop_weekday'] = dropoff_datetime_dim['tpep_dropoff_datetime'].dt.weekday

    dropoff_datetime_dim['dropoff_datetime_id'] = dropoff_datetime_dim.index
    return dropoff_datetime_dim

In [13]:
create_dropoff_datetime_dim(df).head()

,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday,dropoff_datetime_id
0,2024-01-01 01:17:43,1,1,1,2024,0,0
1,2024-01-01 00:09:36,0,1,1,2024,0,1
2,2024-01-01 00:35:01,0,1,1,2024,0,2
3,2024-01-01 00:44:56,0,1,1,2024,0,3
4,2024-01-01 00:52:57,0,1,1,2024,0,4


#### rate_code_dim

In [14]:
def create_rate_code_dim():
    rate_code_type = {
        1:"Standard rate",
        2:"JFK",
        3:"Newark",
        4:"Nassau or Westchester",
        5:"Negotiated fare",
        6:"Group ride"
    }
    
    rate_code_dim = pd.DataFrame(list(rate_code_type.items()), columns=['rate_code_id', 'rate_code_name'])
    return rate_code_dim

In [15]:
create_rate_code_dim()

,rate_code_id,rate_code_name
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


#### payment_type_dim

In [16]:
def create_payment_type_dim():
    payment_type = {
        1:"credit card",
        2:"case",
        3:"no charge",
        4:"dispute",
        5:"unknown",
        6:"voided trip"
    }
    
    payment_type_dim = pd.DataFrame(list(payment_type.items()), columns=['payment_type_id', 'payment_type_name'])
    return payment_type_dim

In [17]:
create_payment_type_dim()

,payment_type_id,payment_type_name
0,1,credit card
1,2,case
2,3,no charge
3,4,dispute
4,5,unknown
5,6,voided trip


In [18]:
create_payment_type_dim().shape

(6, 2)

### Create Fact Table

In [44]:
pickup_datetime_dim = create_pickup_datetime_dim(df)
dropoff_datetime_dim = create_dropoff_datetime_dim(df)
def create_fact_table(df, rate_code_dim, pickup_datetime_dim, dropoff_datetime_dim, payment_type_dim, location_dim):
    fact_table = df.merge(rate_code_dim, left_on='RatecodeID', right_on='rate_code_id', how='left') \
                   .merge(pickup_datetime_dim, left_on='tpep_pickup_datetime', right_on='tpep_pickup_datetime', how='left') \
                   .merge(dropoff_datetime_dim, left_on='tpep_dropoff_datetime', right_on='tpep_dropoff_datetime', how='left') \
                   .merge(payment_type_dim, left_on='payment_type', right_on='payment_type_id', how='left') \
                   .merge(location_dim, left_on='PULocationID', right_on='LocationID', how='left') \
                   .merge(location_dim, left_on='DOLocationID', right_on='LocationID', how='left') \
                   [['trip_id','VendorID', 'pickup_datetime_id', 'dropoff_datetime_id','passenger_count', 'trip_distance',
                     'rate_code_id', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID',
                     'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
                      'total_amount', 'congestion_surcharge', 'airport_fee']]
    return fact_table

In [45]:
fact_table = create_fact_table(df, rate_code_dim, pickup_datetime_dim, dropoff_datetime_dim, payment_type_dim, location_dim)
fact_table

,trip_id,VendorID,pickup_datetime_id,dropoff_datetime_id,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,PULocationID,DOLocationID,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,total_amount,congestion_surcharge,airport_fee
0,0,1,0,0,2.0,3.80,1.0,N,142,236,1.0,14.50,3.0,0.5,3.65,0.0,21.95,2.5,0.0
1,1,1,1,1,1.0,2.10,1.0,N,236,42,1.0,8.00,0.5,0.5,4.00,0.0,13.30,0.0,0.0
2,2,2,2,2,1.0,0.97,1.0,N,166,166,1.0,7.50,0.5,0.5,1.76,0.0,10.56,0.0,0.0
3,3,2,3,3,1.0,1.09,1.0,N,114,68,2.0,8.00,0.5,0.5,0.00,0.0,11.80,2.5,0.0
4,4,2,4,4,1.0,4.30,1.0,N,68,163,1.0,23.50,0.5,0.5,3.00,0.0,30.30,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2463926,2463926,2,1400415,1424264,NaN,1.32,NaN,None,90,170,NaN,8.00,0.0,0.5,2.39,0.0,13.69,NaN,NaN
2463927,2463927,2,1399145,1399787,NaN,4.19,NaN,None,107,75,NaN,16.80,0.0,0.5,4.35,0.0,24.45,NaN,NaN
2463928,2463928,2,1400269,1424251,NaN,2.10,NaN,None,113,246,NaN,11.22,0.0,0.5,2.00,0.0,16.52,NaN,NaN
2463929,2463929,2,1400320,1400772,NaN,2.92,NaN,None,148,164,NaN,12.40,0.0,0.5,0.00,0.0,15.70,NaN,NaN


### Inspect 2022-24 data

In [48]:
folder_path = "data/yellow_tripdata/"
all_file_paths = [os.path.join(folder_path, file_name) for file_name in os.listdir(folder_path)]


zones_path = 'data/taxi_zones/taxi_zones.shp'
zone_lookup_path = 'data/taxi_zone_lookup.csv'
location_dim = create_location_dim(zones_path, zone_lookup_path)
rate_code_dim = create_rate_code_dim()
payment_type_dim = create_payment_type_dim()


for yellow_path in all_file_paths:
    df = read_yellow_data(yellow_path)
    try: 
        pickup_datetime_dim = create_pickup_datetime_dim(df)
    except:
        print(f"{yellow_path} pickup_datetime_dim error")
    try:
        dropoff_datetime_dim = create_dropoff_datetime_dim(df)
    except:
        print(f"{yellow_path} dropoff_datetime_dim error")
    try:
        fact_table = create_fact_table(df, rate_code_dim, pickup_datetime_dim, dropoff_datetime_dim, payment_type_dim, location_dim)
        month_str = os.path.basename(yellow_path).replace("yellow_tripdata_", "").replace(".parquet", "")
        print(f"process {month_str} successful")
        print(f"original vs fact_table shape? --> {df.shape[0] == fact_table.shape[0]}")
    except:
        print(f"{yellow_path} fact_table error")
    

process 2023-06 successful
original vs fact_table shape? --> True
process 2022-10 successful
original vs fact_table shape? --> True
process 2022-09 successful
original vs fact_table shape? --> True
process 2024-04 successful
original vs fact_table shape? --> True
process 2022-08 successful
original vs fact_table shape? --> True
process 2024-05 successful
original vs fact_table shape? --> True
process 2023-07 successful
original vs fact_table shape? --> True
process 2022-01 successful
original vs fact_table shape? --> True
process 2022-11 successful
original vs fact_table shape? --> True
process 2024-07 successful
original vs fact_table shape? --> True
process 2023-05 successful
original vs fact_table shape? --> True
process 2022-03 successful
original vs fact_table shape? --> True
process 2023-04 successful
original vs fact_table shape? --> True
process 2022-02 successful
original vs fact_table shape? --> True
process 2022-12 successful
original vs fact_table shape? --> True
process 20